In [2]:
# libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [10]:
# read data
business_data = pd.read_csv('business_coordinates.csv')
tree_df = pd.read_csv('Tree_Inventory_20240929.csv')

Clean up tree data for processing and rename some columns (longitude) and drop some unnecessary / repetitive ones:

In [11]:
# Process tree data csv
tree_data = tree_df

# get proper latitude and longitude data
from shapely.wkt import loads

# Convert WKT to shapely Point and extract latitude and longitude
tree_data['geometry'] = tree_data['GEOMETRY'].apply(loads)
tree_data['LATITUDE'] = tree_data['geometry'].apply(lambda point: point.y)
tree_data['LONGITUDE'] = tree_data['geometry'].apply(lambda point: point.x)

# drop unnecessary columns
tree_data = tree_data.drop(columns=['geometry', 'LONGTITUDE', 'New Georeferenced Column'])
tree_data['SPECIES'] = tree_data['SPECIES'].astype(str)

tree_data.head()

Getting the trees within a specified radius (set in kilometers) for each business - this one is slow

In [ ]:
# Set radius for businesses to find local trees
from geopy.distance import geodesic
total_businesses = len(business_data)

radius = 1 # 1 km or other distance

# for each business, use the coordinates to get nearby trees within the specified radius
for idx, business_row in business_data.iterrows():

    # get location of business
    business_location = (business_row['lat'], business_row['long'])
    
    # initialize tree list for each business
    nearby_trees = []

    # for each tree, check if it is within the radius
    for _, tree_row in tree_data.iterrows():
        tree_location = (tree_row['LATITUDE'], tree_row['LONGITUDE'])
        distance = geodesic(business_location, tree_location).km

        # append within radius
        if distance <= radius:
            nearby_trees.append(tree_row['SPECIES'])
            
    # Store species of nearby trees
    business_data.at[idx, 'NearbyTreeSpecies'] = ', '.join(nearby_trees)

    # Print progress
    print(f"Processed business {idx + 1} of {total_businesses}")


Faster way to get distances by converting stuff to radians? 

In [23]:
from sklearn.neighbors import BallTree
import numpy as np

# Radius in kilometers
radius = 1.0 / 6371.0  # Earth radius in km for haversine

# Prepare data for BallTree in radians
business_coords = np.radians(business_data[['lat', 'long']].values)
tree_coords = np.radians(tree_data[['LATITUDE', 'LONGITUDE']].values)

# Create BallTree for the tree locations
tree_ball_tree = BallTree(tree_coords, metric='haversine')

# Function to find nearby tree species for each business
def find_nearby_trees_for_business(business_location):
    # Query the tree BallTree for points within the radius
    indices = tree_ball_tree.query_radius([business_location], r=radius)[0]
    nearby_trees = tree_data.iloc[indices]['SPECIES'].tolist()
    return ', '.join(nearby_trees)

# Apply the function to each business
business_data['NearbyTreeSpecies'] = [find_nearby_trees_for_business(loc) for loc in business_coords]

# Optional: Print progress
print("Completed processing for all businesses.")


Completed processing for all businesses.


Save as csv if necessary

In [25]:
# save as csv
# business_data.to_csv('business_trees.csv', index=False)

Decision Tree Era

In [63]:
from collections import Counter

# read data (if necessary) and make sure list is string
data = pd.read_csv('business_trees_1km.csv')
data['NearbyTreeSpecies'] = data['NearbyTreeSpecies'].astype(str)

# Function to find the most common species
def get_most_common_species(nearby_species):
    species_list = nearby_species.split(', ')  # Split the string into a list of species
    species_counts = Counter(species_list)  # Count occurrences of each species
    most_common_species, _ = species_counts.most_common(1)[0]  # Get the most common species
    return most_common_species

# Apply the function to create a new column for the predicted species
data['BestTreeSpecies'] = data['NearbyTreeSpecies'].apply(get_most_common_species)

data.head()

,Unnamed: 0,Reported.Year,Property.ID..,Commercial.Property..Property.Street.Address,Building.Name,Total.SqFt,Category,Year.Built,kWh.sqft,Percentile.Rank,...,Portfolio.Manager.SqFt,Total.Green.House.Gas.Emissions..MtCO2e.,Portfolio.Manager.Organization,GeoLocation,Submitted.Name.of.Organization,GeoLocation_clean,lat,long,NearbyTreeSpecies,BestTreeSpecies
0,1,2017.0,187746,"5126 E 5 ST, AUSTIN TX,","5126 E 5 ST, AUSTIN TX,",7419.0,NaN,1940,0.000,NaN,...,NaN,NaN,NaN,"5126 E 5 ST\nAUSTIN, TX\n(30.251707, -97.697313)","Pantur, Inc.","30.251707, -97.697313",30.251707,-97.697313,"Hackberry, Oak, Live (Southern), Hackberry, Oa...",Pecan
1,2,2017.0,230740,"2500 RIDGEPOINT DR, AUSTIN TX, 78728","2500 RIDGEPOINT DR, AUSTIN TX, 78728",49024.0,INDUSTRIAL (W/sqft > 3),1983,15.580,39.58,...,NaN,NaN,NaN,"2500 RIDGEPOINT DR\nAUSTIN, TX 78728\n(30.3280...",Equitable Commercial Realty,"30.328069, -97.672401",30.328069,-97.672401,nan,nan
2,3,2017.0,729528,"1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741","1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741",101953.0,STORE\GROCERY (W/sqft > 5),2009,32.063,76.26,...,NaN,NaN,NaN,"1700 S PLEASANT VALLEY RD\nAUSTIN, TX 78741\n(...","Mimco, Inc.","30.236494, -97.721013",30.236494,-97.721013,"Oak, Texas Live (Escarpment), Crapemyrtle, Cra...",Oak
3,5,2017.0,526381,2011 W RUNDBERG LN Austin TX 78758,2011 W RUNDBERG LN Austin TX 78758,146142.0,OFFICE LG > 35000,1999,8.049,4.79,...,NaN,NaN,NaN,"2011 W RUNDBERG LN\nAustin, TX 78758\n(30.3737...",AISD,"30.373732, -97.719236",30.373732,-97.719236,"Oak, Texas Red, Oak, Post, Pecan, Sycamore, Me...",Southern Live Oak
4,6,2017.0,200001,"202 W 17 ST, AUSTIN TX, 78701","202 W 17 ST, AUSTIN TX, 78701",15350.0,OFFICE MED 10-35,1998,100.129,NaN,...,NaN,NaN,NaN,"202 W 17 ST\nAUSTIN, TX 78701\n(30.279196, -97...",Charter Communications,"30.279196, -97.740475",30.279196,-97.740475,"Chinese Tallow, Escarpment Live Oak, Pecan, Va...",Pecan


encode for categories

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder


# Encode categorical features, including the target if it's categorical
encoder = LabelEncoder()
data['Category_encoded'] = encoder.fit_transform(data['Category'])
data['BestTreeSpecies_encoded'] = encoder.fit_transform(data['BestTreeSpecies'])


fix na's (right now it's just removing but maybe later imputing would be better)

In [65]:
data.head()

# get number and proportion of nas in the features and targets
print(data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'BestTreeSpecies_encoded']].isna().sum())
print(data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'BestTreeSpecies_encoded']].isna().mean())

# maybe a future improvement is imputing kWH.sqft but for now i'm just dropping the rows with nas
data = data.dropna(subset=['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'BestTreeSpecies_encoded'])
print(data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'BestTreeSpecies_encoded']].isna().sum())
print(data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'BestTreeSpecies_encoded']].isna().mean())

Total.SqFt                   6
Category_encoded             0
Year.Built                  64
kWh.sqft                   827
lat                          0
long                         0
BestTreeSpecies_encoded      0
dtype: int64
Total.SqFt                 0.002137
Category_encoded           0.000000
Year.Built                 0.022792
kWh.sqft                   0.294516
lat                        0.000000
long                       0.000000
BestTreeSpecies_encoded    0.000000
dtype: float64
Total.SqFt                 0
Category_encoded           0
Year.Built                 0
kWh.sqft                   0
lat                        0
long                       0
BestTreeSpecies_encoded    0
dtype: int64
Total.SqFt                 0.0
Category_encoded           0.0
Year.Built                 0.0
kWh.sqft                   0.0
lat                        0.0
long                       0.0
BestTreeSpecies_encoded    0.0
dtype: float64


In [68]:
# Define features and target
features = data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long']]
target = data['BestTreeSpecies_encoded']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

'''# Initialize and train the decision tree
clf = DecisionTreeClassifier(criterion='gini')  # Use 'entropy' if you'd like to try that
clf.fit(X_train, y_train)

# Evaluate accuracy
accuracy = clf.score(X_test, y_test)
print(f"Decision Tree Accuracy: {accuracy * 100:.2f}%")'''


'# Initialize and train the decision tree\nclf = DecisionTreeClassifier(criterion=\'gini\')  # Use \'entropy\' if you\'d like to try that\nclf.fit(X_train, y_train)\n\n# Evaluate accuracy\naccuracy = clf.score(X_test, y_test)\nprint(f"Decision Tree Accuracy: {accuracy * 100:.2f}%")'